# Census data summaries for 2000, 2010, 2020 census

Summaries for 2010 and 2020 are derived from the [American Community Survey](https://www.census.gov/programs-surveys/acs). Year 2000 summaries are derived from the 2000 decentenial census. Tract level socio-economic data is summarized for three geographies: 1-mile radius, closest tracts, tenderloin/SoMA neighborhoods. 

## Import packages

In [252]:
# base libraries
import requests, json, os
import pandas as pd
import numpy as np
from collections import defaultdict

# graph libraries
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Retrieve data from Census API
All socio-economic data comes from the Census ACS 5-year estimates for 2010 and 2020 and the 2000 decentenial census and is available at the tract level through the census API. See census API docs [here](https://www.census.gov/data/developers/data-sets.html)

### Census Attribute IDs
The census API returns attribute values for provided attribute IDs. A list of attribute ID's needed for calculating the socio-economic profile data is compiled below from IDs stored in attribute_lookup.csv.

In [253]:
#set run year
run_year = "2000"

In [254]:
# create list of attribute IDs from attribute_lookup.csv
attribute_lookup_df = pd.read_csv (r'./lookup_tables/attribute_lookup.csv', dtype=str)

In [255]:
# set year and extension to choose API endpoint
attribute_col = "attribute_" + run_year
extension_col = "extension_" + run_year

attribute_dict = attribute_lookup_df[[attribute_col, extension_col]].to_dict('records')

attribute_dict_source = defaultdict(list)
for record in attribute_dict:
    attribute_dict_source[record[extension_col]]+=record[attribute_col].split(",")  

### Build Census API URL and Make Query
The code below builds the URL for the census API call 

- Tract code is '*' to collect all tracts
- State code is '06' for CA
- County code is '075' for San Francisco County
- Attributes are defined by the attribute id list and includes all relevant attributes for the socio-economic data calcs

In [256]:
census_endpoints = {"2000": r"2000/dec/", 
                   "2010":r"2010/acs/", 
                   "2020":r"2020/acs/"}

In [257]:
# function builds the api URL from tract_code, state_code, county_code, and attribute ids. 
def build_census_url(tract_code, state_code, county_code, attribute_ids, file_name):
    attributes = ','.join(attribute_ids)
    census_url = r'https://api.census.gov/data/'+ census_endpoints[run_year] + file_name + '?get={}&for=tract:{}&in=state:{}&in=county:{}'\
                .format(attributes, tract_code, state_code, county_code)
    return census_url
    

In [258]:
# function makes a single api call and collects results in a pandas dataframe
def make_census_api_call(census_url):
    # make API call to Census
    resp = requests.get(census_url)
    if resp.status_code != 200:
        # this means something went wrong
        resp.raise_for_status()
       
    # retrieve data as json and convert to Pandas Dataframe
    data = resp.json()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)

    # convert values that are not state, county, or tract to numeric type
    cols=[i for i in df.columns if i not in ["state","county","tract"]]
  
    return df

In [259]:
# set geo variables and make api call
tract_code = "*"
state_code = "06"
county_code = "075"

df=None
first = True

for file in attribute_dict_source.keys():
    attribute_ids = attribute_dict_source[file]
    attribute_ids = list(set(attribute_ids))
    # split attributes into groups of 45, run a census query for each, merge outputs into a single df
    split_attribute_ids = [attribute_ids[i:i+45] for i in range(0, len(attribute_ids), 45)]

    for ids in split_attribute_ids:
        census_url = build_census_url(tract_code, state_code, county_code, ids, file)
        #census_url = build_census_url(state_code, county_code, ids, file)
        print(census_url)
        returned_df = make_census_api_call(census_url)
        if first:
            df = returned_df
            first = False
        else:
            returned_df = returned_df.drop(columns=['state', 'county'])
            df = pd.merge(df, returned_df, on='tract', how='left')



https://api.census.gov/data/2000/dec/sf1?get=P015B001,P015D001,P007005,P007007,P007003,P015H001,P015G001,P009004,P019002,P015I001,P015F001,P008010,P008003,P023002,P007008,P007001,P007006,P015E001,P015001,P015C001&for=tract:*&in=state:06&in=county:075
https://api.census.gov/data/2000/dec/sf3?get=HCT011020,H069010,H011010,HCT011023,P042004,P042038,P042024,HCT011018,H008002,HCT011024,P092002,HCT011003,H069009,HCT011021,P042007,HCT012003,H069007,H069008,P042031,HCT011019,P042045,P042028,P042021,HCT011022,H001001,H011002,H006003,P042048,HCT011025,HCT011016,HCT011017,H016002,P042015&for=tract:*&in=state:06&in=county:075


In [260]:
# set tract to int
df["tract"]=df["tract"].astype(str).astype(int)

## Prep geo lookup tables before to subset according to 3 geographies

In [261]:
# Create list of attribute IDs from attribute_lookup.csv
geo_lookup_df = pd.read_csv (r'./lookup_tables/geo_lookup_' + run_year + '.csv', dtype=str)

In [262]:
geo_lookup_df["tractid"]= geo_lookup_df["tractid"].astype(str).astype(int)

In [264]:
closest_tracts = geo_lookup_df[geo_lookup_df["closest"]=="Y"]["tractid"].tolist()
df_closest=df[df["tract"].isin(closest_tracts)]
df_closest=df_closest.set_index('tract')
df_closest_t=df_closest.T
df_closest_t = df_closest_t.reset_index()
df_closest_t = df_closest_t.rename(columns={"tract": "", "index": "attribute"})
df_closest_t = df_closest_t.reset_index()

In [265]:
mile_1_tracts = geo_lookup_df[geo_lookup_df["1mile"]=="Y"]["tractid"].tolist()
df_mile_1=df[df["tract"].isin(mile_1_tracts)]
df_mile_1=df_mile_1.set_index('tract')
df_mile_1_t =df_mile_1.T
df_mile_1_t = df_mile_1_t.reset_index()
df_mile_1_t = df_mile_1_t.rename(columns={"tract": "", "index": "attribute"})
df_mile_1_t = df_mile_1_t.reset_index()

In [266]:
neighborhood_tracts = geo_lookup_df[geo_lookup_df["neighborhood"].isin(["Tenderloin","SOMA"])]["tractid"].tolist()
df_neighborhood = df[df["tract"].isin(neighborhood_tracts)]
df_neighborhood=df_neighborhood.set_index('tract')
df_neighborhood_t = df_neighborhood.T
df_neighborhood_t = df_neighborhood_t.reset_index()

df_neighborhood_t = df_neighborhood_t.rename(columns={"tract": "", "index": "attribute"})
df_neighborhood_t = df_neighborhood_t.reset_index()

## Define functions for calculating socio-economic data
Takes care of attributes that are calculated as a sum of multiple attributes

In [268]:
attributes = attribute_lookup_df.to_dict('records')

In [269]:
# function runs all calcs for each neighborhood or supervisor district
def calc_socio_economic_data(df_t,tracts, attributes):
    # create empty dictionary to add calculated attribute information to
    all_calc_data = []
    # calculate all stats for each neighborhood
    for attribute in attributes:
        attribute_info = {'category':attribute['category'], "attribute_name":attribute["attribute_name"]}
        # population
        attribute_list = attribute[attribute_col].split(",")
        filter_values = df_t[df_t['attribute'].isin(attribute_list)]
        for tract in tracts:
            attribute_value = filter_values[tract].astype(str).astype(int).sum()
            attribute_info[tract] = attribute_value 
      
        all_calc_data.append(attribute_info)
        
            
    #return calc dictionary
    return all_calc_data

## Caculate Socioeconomic Profiles

### Run Socioeconomic Profiles Calcs

In [270]:
# calculate 1 mile data
mile_1_data = calc_socio_economic_data(df_mile_1_t, mile_1_tracts, attributes)
mile_1_data_df = pd.DataFrame(mile_1_data)

In [271]:
# calcualte neighborhood data
neighborhood_data = calc_socio_economic_data(df_neighborhood_t, neighborhood_tracts, attributes)
neighborhood_data_df = pd.DataFrame(neighborhood_data)

In [272]:
# calculate closest data
closest_data = calc_socio_economic_data(df_closest_t, closest_tracts, attributes)
closest_data_df = pd.DataFrame(closest_data)

## Export

In [273]:
# export both dataset views to csv
neighborhood_data_df.to_csv(r"./output/neighborhood_"+run_year+".csv", index = False)

In [274]:
mile_1_data_df.to_csv(r"./output/mile_1_"+run_year+".csv", index = False)

In [275]:
closest_data_df.to_csv(r"./output/closest_"+run_year+".csv", index = False)